![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [1]:
from IPython.display import Image, display, clear_output
import pydot
import sys
import time
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the VMK180 TRD notebook 2 (nb2).

In [2]:
nb = "nb2"

In [3]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

env: GST_DEBUG_DUMP_DOT_DIR=/home/root/gst-dot/nb2


In [4]:
import xlnx_pciesrc
import ctypes
from ctypes import *

In [5]:
Gst.init(None)

pcie_fd = xlnx_pciesrc.PCIe_GetDevice()
usecase = xlnx_pciesrc.PCIe_Getusecase(pcie_fd)
print(usecase)

if usecase.value == 1 :
        src_type = "mipi"
        io_mode = "dmabuf"
        src = Gst.ElementFactory.make("xlnxvideosrc")
        src.set_property("io-mode", io_mode)
        src.set_property("src-type", src_type)
        res_dict = {
            "1080p" : ("1920", "1080"),
            "2160p" : ("3840", "2160")
            }
        res = "2160p" # Change the resolution string to 720p, 1080p, or 2160p (mipi only)
        width = res_dict[res][0]
        height = res_dict[res][1]
        print("Selected resolution: " + width + "x" + height)

        fmt = "YUY2"

        caps = Gst.ElementFactory.make("capsfilter")
        cap = Gst.Caps.from_string("video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt)
        caps.set_property("caps", cap)

elif ((usecase.value > 1) & (usecase.value <= 4) ) :
        print("pcie_src usecase")
        src = Gst.ElementFactory.make("appsrc")
        caps = Gst.ElementFactory.make("capsfilter")
        xlnx_pciesrc.xlnx_pcieappsrc(src,caps)
        xlnx_pciesrc.export_pciedmabuff(pcie_fd)
        
else :
        print("invalid use case")
        exit()

c_int(4)
pcie_src usecase
video/x-raw, width=3840, height=2160, format=YUY2, framerate=30/1


In [6]:
if ((usecase.value >= 1) & (usecase.value <=3)) :
        sink = Gst.ElementFactory.make("appsink")
        xlnx_pciesrc.xlnx_pcieappsink(sink)
elif (usecase.value == 4)   :
        print("kmssink use case")
        driver_name = "xlnx"
        plane_id = 38
        xoff = 0 # Change this value to move the plane position in the x-direction
        yoff = 0 # Change this value to move the plane position in the y-direction
        width = int('3840', 10)
        height = int('2160', 10)
        render_rectangle = Gst.ValueArray((xoff, yoff, width, height))
        sink = Gst.ElementFactory.make("kmssink")
        sink.set_property("driver-name", driver_name)
        sink.set_property("plane-id", plane_id)
        sink.set_property("render-rectangle", render_rectangle)
        sink.set_property("sync", False)
else :
        print("invalid use case")
        exit()

kmssink use case


# 5. Create and Run the GStreamer Pipeline

In [7]:
#add ivas filter as well
add_filter = '0'

In [8]:
#add ivas filter as well
if add_filter == '1' :
    jsondir = "/usr/share/vvas/vmk180-trd/"
    filter_kernels = ["PL"]
    filter_kernel = filter_kernels[0] # Change filter kernel to PL, AIE or SW via list index
    print("Selected filter2d kernel: " + filter_kernel)

    if filter_kernel == "PL":
        jfile = jsondir + "kernel_xfilter2d_pl.json"
        #"kernel_xfilter2d_pl_lk.json"
    elif filter_kernel == "AIE":
        assert width == 1280 and height == 720, "The AIE filter2d kernel only supports 720p!"
        # TODO
    else: # filter_kernel == "SW"
        jfile = jsondir + "kernel_xfilter2d_sw.json"

    filter2d = Gst.ElementFactory.make("vvas_xfilter")
    filter2d.set_property("kernels-config", jfile)

    plist = [
        "blur",
        "edge",
        "horizontal edge",
        "vertical edge",
        "emboss",
        "horizontal gradient",
        "vertical gradient",
        "identity",
        "sharpen",
        "horizontal sobel",
        "vertical sobel",
        "custom"
    ]

    def print_presets():
        print("Supported filter presets:\n")
        print('\n'.join(plist) + '\n')
    
    #print_presets()

    def set_preset(val):
        if val in plist:
            jstring = '{ "filter_preset" : "' +  val + '" }'
        #print(jstring)
            filter2d.set_property("dynamic-config", jstring)
        else:
            raise Exception("Unsupported filter preset \'" + val + "\'")

    set_preset("identity")

    def set_coeff(val):
        jstring = '{ "filter_coefficients" : ' + val + ' }'
        print(jstring)
        filter2d.set_property("dynamic-config", jstring)

perf =  Gst.ElementFactory.make("perf")

Create the pipeline, add all elements, and link them together.

In [9]:
pipeline = Gst.Pipeline.new(nb)
pipeline.add(src)
pipeline.add(caps)
pipeline.add(perf)
if ( usecase.value == 1 ) :
    pipeline.add(caps)
#pipeline.add(perf)
pipeline.add(sink)

if(usecase.value == 1) :
    src.link(caps)
    caps.link(sink)
else :
    src.link(caps)
    caps.link(perf)
    perf.link(sink)


Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. The frame rate will be printed and updated below the code cell.

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [10]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        xlnx_pciesrc.xlnx_pciecleanup()
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.INFO:
        err, info = message.parse_info()
        sys.stderr.write("Info: %s\n" % info)
        clear_output(wait=True)
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

In [11]:
pipeline.set_state(Gst.State.PLAYING);
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message",bus_call, loop)
try:
    loop.run()
except:
    if (usecase.value == 1) :
            xlnx_pciesrc.xlnx_pciecleanup()
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

Interrupt caught


/tmp/ipykernel_814/3250633036.py:12: Warning: g_value_set_boolean: assertion 'G_VALUE_HOLDS_BOOLEAN (value)' failed
  Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)


# 7. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device and display it on a monitor
2. Plot the live memory bandwidth by running the APM notebook in parallel
3. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>